
# Fixing the BART Track Lines GeoJSON file
## We want the BART track Lines, but we need to fix JSON file first to use its geometry

## Imports

In [1]:
import math
import numpy as np
import pandas as pd
import random

import psycopg2

import json

import gmaps
import gmaps.geojson_geometries

from geographiclib.geodesic import Geodesic

## my_select_query_pandas() - function to run a select query and return rows in a Pandas dataframe

In [12]:
#
# function to run a select query and return rows in a pandas dataframe
# pandas puts all numeric values from postgres to float
# if it will fit in an integer, change it to integer
#

def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)
    

## Connect to the Postgres database

In [13]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

## Create a cursor for the connection

In [4]:
cursor = connection.cursor()

## Connect to Google Maps using your api key;  edit the file gmap_api_key.txt and put in your api key

In [5]:
f = open('gmap_api_key.txt', 'r')
my_api_key = f.read()
f.close()

gmaps.configure(api_key=my_api_key)

## Basic map centered on Sather Gate at UC Berkeley; all point are (latitude, longitude) in decimal

In [6]:
sather_gate_berkeley = (37.870260430419115, -122.25950168579497)

# gmaps.figure(center=sather_gate_berkeley, zoom_level=9)

## Project 3 Continuation

In [ ]:
import json
f = open( "BART_Track2.geojson" , "rb" ) #here we can change the file
jsonObject = json.load(f)
f.close()
jsonObject

### This is the structure it should have after editing

In [ ]:
import json
f = open( "BART_Track_edited5.geojson" , "rb" )
jsonObject2 = json.load(f)
f.close()
jsonObject2

## Inserting the first bracket of the coordinate pair

In [ ]:
import re
import string
String = "I am your father"
iter = re.finditer("fat", String)
indices = [m.start(0) for m in iter]


json_string = json.dumps(jsonObject)

iter = re.finditer("coordinates", json_string)
coordinates = [m.start(0) for m in iter] # all the places where coordinates exists 

iter = re.finditer("]}}", json_string) 
group_end = [m.start(0) for m in iter] #all the places where type feature section ends "]}}"
group_end

iter = re.finditer("\[-12", json_string) 
coord_pair_start = [m.start(0) for m in iter] #all the places where type feature section ends "]}}"
coord_pair_start

aStr = json_string
n = len(coord_pair_start)-1
i = 0
while (i <= n):
    #print("i before update", i)
    
    pos = coord_pair_start[i]
    aStr = aStr[:pos] + "[[" + aStr[pos+1:] #does adding a line break work?
    i += 2 
    #print("i after update", i)
  
    iter = re.finditer("\[-12", aStr) 
    coord_pair_start = [m.start(0) for m in iter] #all the places where type feature section ends "]}}"

aStr



## inserting the last bracket

In [ ]:


iter = re.finditer("0.0\]", aStr) 
coord_pair_end = [m.start(0) for m in iter] #all the places where type feature section ends "]}}"
n = len(coord_pair_end)-1
i = 1
print("beggining of last while loop")
while (i < n):
    #print("i before update", i)    
    pos = coord_pair_end[i]
    aStr = aStr[:pos + 3] + "]]" + aStr[pos+3+1:]    i += 2 
    #print("i after update", i)  
    iter = re.finditer("0.0\]", aStr) 
    coord_pair_end = [m.start(0) for m in iter] #all the places where coordinate pair section ends "0.0\]"
aStr

### Need remove all of the 0.0] and replace LineString with MultiLineString

In [ ]:
cStr = aStr.replace(', 0.0', '')
cStr
dStr = cStr.replace('LineString', 'MultiLineString')
dStr

### Lets write to a file, final string is dStr

In [ ]:
f = open("BART_Track_edited9.geojson", "x")
f.write(dStr)
f.close()

### Lets Load Final Json File
### But adding color scheme for each BART Line
#### Total 11 Lines, but color is not loading

In [32]:
#BART_Track_edited10.geojson is the json file that is well constructed
with open('BART_Track_edited10.geojson') as f:
    geometry = json.load(f)
colors = ['red', 'black', 'blue', 'purple', 'black', 'green', 'blue', 'purple', 'red', 'black', 'blue']
fig = gmaps.figure()
geojson_layer = gmaps.geojson_layer(geometry, stroke_color=colors)
fig.add_layer(geojson_layer)
fig

Figure(layout=FigureLayout(height='420px'))

## Overlay of BART stations with Plain Dots

In [29]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select * from stations



"""
stations_df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
colors = []
for i in range(0,50):
    colors.append(np.random.choice(['red', 'green', 'blue', 'purple', 'yellow', 'teal']))

#stations_df.head()
#print(station_locations )

#color green is chosen for dollars sign $$$

fig = gmaps.figure(center=sather_gate_berkeley, map_type = 'HYBRID', zoom_level=9)

df_markers = stations_df[['latitude','longitude']]

marker_layer = gmaps.symbol_layer(df_markers, fill_color= colors, stroke_color=colors )

fig.add_layer(marker_layer)

fig


Figure(layout=FigureLayout(height='420px'))

In [28]:
stations_df.head(100)
stations_df.shape
colors = []
for i in range(0,50):
    colors.append(np.random.choice(['red', 'green', 'blue', 'purple', 'yellow', 'teal']))
    
len(colors)

50